## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Codrington,AU,-38.2667,141.9667,55.74,59,66,11.68,broken clouds
1,1,Itaquitinga,BR,-7.6678,-35.1017,81.61,88,25,2.62,scattered clouds
2,2,Port Alfred,ZA,-33.5906,26.8910,56.48,79,0,10.69,clear sky
3,3,Kamenka,RU,51.3223,42.7678,32.05,77,12,13.80,few clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,35.26,69,75,20.71,light snow


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 70


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df= city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
11,11,Ponta Do Sol,PT,32.6667,-17.1000,68.72,63,62,7.65,broken clouds
14,14,Polis,CY,35.0367,32.4264,60.62,92,3,1.92,clear sky
23,23,Huarmey,PE,-10.0681,-78.1522,63.52,84,63,10.16,broken clouds
25,25,Sakakah,SA,29.9697,40.2064,63.97,34,0,6.73,clear sky
26,26,Lagoa,PT,39.0500,-27.9833,66.38,73,58,6.82,broken clouds
...,...,...,...,...,...,...,...,...,...,...
682,682,Rurrenabaque,BO,-14.4413,-67.5278,62.87,96,100,3.91,overcast clouds
683,683,Buraydah,SA,26.3260,43.9750,66.65,32,0,5.26,clear sky
690,690,Vallenar,CL,-28.5708,-70.7581,65.37,50,0,3.74,clear sky
709,709,Lavumisa,SZ,-27.3100,31.8920,65.52,77,86,3.36,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                117
City                   117
Country                115
Lat                    117
Lng                    117
Max Temp               117
Humidity               117
Cloudiness             117
Wind Speed             117
Current Description    117
dtype: int64

In [7]:
preferred_cities_df.notnull().sum()

City_ID                117
City                   117
Country                115
Lat                    117
Lng                    117
Max Temp               117
Humidity               117
Cloudiness             117
Wind Speed             117
Current Description    117
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Ponta Do Sol,PT,68.72,broken clouds,32.6667,-17.1000,
14,Polis,CY,60.62,clear sky,35.0367,32.4264,
23,Huarmey,PE,63.52,broken clouds,-10.0681,-78.1522,
25,Sakakah,SA,63.97,clear sky,29.9697,40.2064,
26,Lagoa,PT,66.38,broken clouds,39.0500,-27.9833,
27,Saint George,US,66.94,clear sky,37.1041,-113.5841,
29,Ponta Delgada,PT,61.52,few clouds,37.7333,-25.6667,
43,Nouadhibou,MR,68.45,overcast clouds,20.9310,-17.0347,
50,Lloret De Mar,ES,62.91,clear sky,41.6999,2.8457,
56,Laguna,US,60.85,heavy intensity rain,38.4210,-121.4238,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Ponta Do Sol,PT,68.72,broken clouds,32.6667,-17.1000,Hotel do Campo
14,Polis,CY,60.62,clear sky,35.0367,32.4264,Marion
23,Huarmey,PE,63.52,broken clouds,-10.0681,-78.1522,Hotel Miramar
25,Sakakah,SA,63.97,clear sky,29.9697,40.2064,Fakhamat Al Orjoana Apartment
26,Lagoa,PT,66.38,broken clouds,39.0500,-27.9833,Casa Das Faias
27,Saint George,US,66.94,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
29,Ponta Delgada,PT,61.52,few clouds,37.7333,-25.6667,Vila Nova Hotel
43,Nouadhibou,MR,68.45,overcast clouds,20.9310,-17.0347,El Medina
50,Lloret De Mar,ES,62.91,clear sky,41.6999,2.8457,Apartaments Guitart
56,Laguna,US,60.85,heavy intensity rain,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))